# Tratamento

### Importações/ Carregamento dados

In [12]:
# Importações necessárias
import pandas as pd

# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV
    dados = pd.read_csv(caminho_arquivo)
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados

# Exemplo de uso da função com coluna de datas 'date'
caminho_arquivo = '../data/raw/AMZN_1997-05-15_2025-02-21.csv'
dados = carregar_dados(caminho_arquivo, coluna_data='date')

# Exibir as primeiras linhas do DataFrame
dados.head()

,date,open,high,low,close,adj_close,volume
0,1997-05-15 04:00:00+00:00,0.121875,0.125000,0.096354,0.097917,0.097917,1443120000
1,1997-05-16 04:00:00+00:00,0.098438,0.098958,0.085417,0.086458,0.086458,294000000
2,1997-05-19 04:00:00+00:00,0.088021,0.088542,0.081250,0.085417,0.085417,122136000
3,1997-05-20 04:00:00+00:00,0.086458,0.087500,0.081771,0.081771,0.081771,109344000
4,1997-05-21 04:00:00+00:00,0.081771,0.082292,0.068750,0.071354,0.071354,377064000


### Tamanho da Base

In [13]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 6986, 'Número de Colunas': 7}

### Verificação dos dados

In [14]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,date,"datetime64[ns, UTC]"
1,open,float64
2,high,float64
3,low,float64
4,close,float64
5,adj_close,float64
6,volume,int64


### Verificação Valores Nulos

In [15]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [16]:
def detectar_valores_duplicados(df):
    """''
    Detecta valores duplicados em um DataFrame e retorna uma mensagem amigável para visualização.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - str ou pandas.DataFrame: Mensagem indicando a presença ou ausência de duplicatas, ou um DataFrame com os detalhes das duplicatas.
    """
    # Verificar duplicatas
    duplicatas = df[df.duplicated(keep=False)]
    
    if duplicatas.empty:
        return "Nenhum valor duplicado foi encontrado nos dados."
    
    # Detalhar duplicatas
    detalhes_duplicatas = []
    for index, row in duplicatas.iterrows():
        for coluna in df.columns:
            if df.duplicated(subset=[coluna], keep=False).iloc[index]:
                detalhes_duplicatas.append({'Linha': index, 'Coluna': coluna, 'Valor': row[coluna]})
    
    return pd.DataFrame(detalhes_duplicatas)

# Detectar valores duplicados
duplicatas = detectar_valores_duplicados(dados)
duplicatas 

'Nenhum valor duplicado foi encontrado nos dados.'

### Verificação consistência dos dados

In [17]:
# Função para verificar a padronização dos dados numéricos
def verificar_padronizacao_numerica(df):
    """
    Verifica a padronização dos dados numéricos em todas as colunas que possuam números.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict ou str: Dicionário com as colunas e os problemas encontrados ou uma mensagem indicando que não foram encontradas inconsistências.
    """
    problemas = {}
    
    for coluna in df.select_dtypes(include=['float64', 'int64']).columns:
        problemas_coluna = []
        
        # Verificar se todos os valores são numéricos
        if not pd.api.types.is_numeric_dtype(df[coluna]):
            problemas_coluna.append("Valores não numéricos encontrados")
        
        # Verificar se os valores são válidos (sem caracteres inválidos)
        if df[coluna].apply(lambda x: isinstance(x, (int, float))).sum() != len(df[coluna]):
            problemas_coluna.append("Valores inválidos encontrados")
        
        # Verificar se os separadores decimais estão corretos
        if df[coluna].apply(lambda x: isinstance(x, float) and '.' not in str(x)).sum() > 0:
            problemas_coluna.append("Separadores decimais incorretos")
        
        if problemas_coluna:
            problemas[coluna] = problemas_coluna
    
    if not problemas:
        return "Nenhuma inconsistência nos dados numéricos foi encontrada."
    
    return problemas
# Verificar a padronização dos dados numéricos
problemas_numericos = verificar_padronizacao_numerica(dados)
problemas_numericos

'Nenhuma inconsistência nos dados numéricos foi encontrada.'

### Validação consistência lógica dos dados

In [18]:
def validar_consistencia_logica(df):
    """
    Valida a consistência lógica dos dados de acordo com as regras de negócio:
    - high ≥ low
    - close dentro do intervalo [low, high]
    - volume não negativo
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com as inconsistências encontradas.
    """
    inconsistencias = []

    # Verificar se high < low
    inconsistencias_high_low = df[df['high'] < df['low']]
    for index, row in inconsistencias_high_low.iterrows():
        inconsistencias.append({'Linha': index, 'Coluna': 'high < low', 'Valor': f"high: {row['high']}, low: {row['low']}"})

    # Verificar se close está fora do intervalo [low, high]
    inconsistencias_close_intervalo = df[(df['close'] < df['low']) | (df['close'] > df['high'])]
    for index, row in inconsistencias_close_intervalo.iterrows():
        inconsistencias.append({'Linha': index, 'Coluna': 'close fora do intervalo [low, high]', 'Valor': f"close: {row['close']}, low: {row['low']}, high: {row['high']}"})

    # Verificar se volume é negativo
    inconsistencias_volume_negativo = df[df['volume'] < 0]
    for index, row in inconsistencias_volume_negativo.iterrows():
        inconsistencias.append({'Linha': index, 'Coluna': 'volume negativo', 'Valor': row['volume']})

    if not inconsistencias:
        return "Nenhuma inconsistência lógica foi encontrada nos dados."

    return pd.DataFrame(inconsistencias)

# Validar a consistência lógica dos dados
inconsistencias_logicas = validar_consistencia_logica(dados)
inconsistencias_logicas

'Nenhuma inconsistência lógica foi encontrada nos dados.'

### Otimização

In [19]:
def otimizar_memoria(df):
    """
    Converte colunas para otimização da memória.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser otimizado.
    
    Retorna:
    - pandas.DataFrame: DataFrame otimizado.
    """
    for coluna in df.select_dtypes(include=['int64']).columns:
        df[coluna] = df[coluna].astype('int32')
    
    return df

# Otimizar a memória do DataFrame
dados_otimizado = otimizar_memoria(dados)
dados_otimizado.dtypes 

date         datetime64[ns, UTC]
open                     float64
high                     float64
low                      float64
close                    float64
adj_close                float64
volume                     int32
dtype: object

### Verificar se dados seguem uma ordem cronológica

In [20]:
def verificar_ordem_cronologica(df, coluna_data):
    """
    Verifica se as datas em uma coluna de um DataFrame estão em ordem cronológica.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    - coluna_data (str): Nome da coluna de datas.
    
    Retorna:
    - bool: True se as datas estiverem em ordem cronológica, False caso contrário.
    """
    return df[coluna_data].is_monotonic_increasing

# Verificar se a coluna 'date' está em ordem cronológica
ordem_cronologica = verificar_ordem_cronologica(dados, 'date')
ordem_cronologica

True

## Formatação - Moeda US$

In [21]:
def arredondar_colunas(df, colunas, casas_decimais=2):
    """
    Arredonda os valores das colunas especificadas para o número de casas decimais fornecido.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame contendo os dados originais.
    - colunas (list): Lista de colunas a serem arredondadas.
    - casas_decimais (int): Número de casas decimais para arredondamento (padrão: 2).

    Retorna:
    - pandas.DataFrame: DataFrame com as colunas arredondadas.
    """
    # Verificar se as colunas existem no DataFrame
    colunas_invalidas = [col for col in colunas if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"As seguintes colunas não existem no DataFrame: {colunas_invalidas}")

    # Criar uma cópia do DataFrame para evitar alterações no original
    df_arredondado = df.copy()

    # Arredondar as colunas especificadas
    for coluna in colunas:
        df_arredondado[coluna] = df_arredondado[coluna].round(casas_decimais)

    # Gerar uma saída amigável
    print(f"As colunas {colunas} foram arredondadas para {casas_decimais} casas decimais.")
    print("\nAs 10 primeiras linhas do DataFrame arredondado são:")
    print(df_arredondado.head(10))

    # Retornar o DataFrame arredondado
    return df_arredondado

# Arredondar as colunas relacionadas à moeda para 2 casas decimais
# Ensure that the correct variables are used
dados = arredondar_colunas(dados, colunas_para_ajustar, casas_decimais=2)


As colunas ['open', 'high', 'low', 'close', 'adj_close'] foram arredondadas para 2 casas decimais.

As 10 primeiras linhas do DataFrame arredondado são:
                       date  open  high   low  close  adj_close      volume
0 1997-05-15 04:00:00+00:00  0.12  0.12  0.10   0.10       0.10  1443120000
1 1997-05-16 04:00:00+00:00  0.10  0.10  0.09   0.09       0.09   294000000
2 1997-05-19 04:00:00+00:00  0.09  0.09  0.08   0.09       0.09   122136000
3 1997-05-20 04:00:00+00:00  0.09  0.09  0.08   0.08       0.08   109344000
4 1997-05-21 04:00:00+00:00  0.08  0.08  0.07   0.07       0.07   377064000
5 1997-05-22 04:00:00+00:00  0.07  0.07  0.07   0.07       0.07   235536000
6 1997-05-23 04:00:00+00:00  0.07  0.08  0.07   0.08       0.08   318744000
7 1997-05-27 04:00:00+00:00  0.08  0.08  0.07   0.08       0.08   173952000
8 1997-05-28 04:00:00+00:00  0.08  0.08  0.08   0.08       0.08    91488000
9 1997-05-29 04:00:00+00:00  0.08  0.08  0.07   0.08       0.08    69456000


## Carga dos dados


In [22]:
def carregar_dados_atualizados(df, caminho_saida):
    """
    Realiza a carga dos dados da nova tabela com todas as atualizações realizadas e salva em um novo arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - caminho_saida (str): Caminho do arquivo CSV de saída.
    
    Retorna:
    - pandas.DataFrame: DataFrame atualizado.
    """
    # Verificar o tamanho da base de dados
    tamanho_base = verificar_tamanho_base(df)
    print("Tamanho da base de dados:", tamanho_base)
    
    # Verificar o tipo dos dados/colunas
    tabela_tipos = verificar_tipo_dados(df)
    print("Tipos de dados das colunas:\n", tabela_tipos)
    
    # Verificar valores ausentes
    tabela_ausentes = verificar_valores_ausentes(df)
    print("Valores ausentes:\n", tabela_ausentes)
    
    # Detectar valores duplicados
    duplicatas = detectar_valores_duplicados(df)
    print("Valores duplicados:\n", duplicatas)
    
    # Verificar a padronização dos dados numéricos
    problemas_numericos = verificar_padronizacao_numerica(df)
    print("Problemas nos dados numéricos:\n", problemas_numericos)
    
    # Validar a consistência lógica dos dados
    inconsistencias_logicas = validar_consistencia_logica(df)
    print("Inconsistências lógicas:\n", inconsistencias_logicas)
    
    # Otimizar a memória do DataFrame
    df_otimizado = otimizar_memoria(df)
    print("Tipos de dados após otimização:\n", df_otimizado.dtypes)
    
    # Verificar se a coluna 'date' está em ordem cronológica
    ordem_cronologica = verificar_ordem_cronologica(df_otimizado, 'date')
    print("Ordem cronológica:", ordem_cronologica)
    
    # Salvar o DataFrame atualizado em um novo arquivo CSV
    df_otimizado.to_csv(caminho_saida, index=False)
    print(f"Dados atualizados salvos em: {caminho_saida}")
    
    return df_otimizado

# Carregar os dados atualizados e salvar em um novo arquivo CSV
caminho_saida = '../data/processed/AMZN_1997-05-15_2025-02-21_atualizado.csv'
dados_atualizados = carregar_dados_atualizados(dados, caminho_saida)
dados_atualizados.head()

Tamanho da base de dados: {'Número de Linhas': 6986, 'Número de Colunas': 7}
Tipos de dados das colunas:
       Coluna         Tipo de Dado
0       date  datetime64[ns, UTC]
1       open              float64
2       high              float64
3        low              float64
4      close              float64
5  adj_close              float64
6     volume                int32
Valores ausentes:
 Empty DataFrame
Columns: [Quantidade de Valores Ausentes, Percentual de Valores Ausentes (%)]
Index: []
Valores duplicados:
 Nenhum valor duplicado foi encontrado nos dados.
Problemas nos dados numéricos:
 Nenhuma inconsistência nos dados numéricos foi encontrada.
Inconsistências lógicas:
 Nenhuma inconsistência lógica foi encontrada nos dados.
Tipos de dados após otimização:
 date         datetime64[ns, UTC]
open                     float64
high                     float64
low                      float64
close                    float64
adj_close                float64
volume                   

,date,open,high,low,close,adj_close,volume
0,1997-05-15 04:00:00+00:00,0.12,0.12,0.10,0.10,0.10,1443120000
1,1997-05-16 04:00:00+00:00,0.10,0.10,0.09,0.09,0.09,294000000
2,1997-05-19 04:00:00+00:00,0.09,0.09,0.08,0.09,0.09,122136000
3,1997-05-20 04:00:00+00:00,0.09,0.09,0.08,0.08,0.08,109344000
4,1997-05-21 04:00:00+00:00,0.08,0.08,0.07,0.07,0.07,377064000
